In [1]:
import findspark
findspark.init('/Users/wy/spark-1.6.0-bin-hadoop2.6')

import pyspark
sc = pyspark.SparkContext(appName="myAppName")

In [9]:
from sklearn import datasets
from math import *
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import cross_validation
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV
from sklearn.cluster import spectral_clustering
from sklearn import mixture
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import decomposition
from sklearn.cross_validation import train_test_split

In [43]:
iris = datasets.load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.25)

clf = RandomForestClassifier(n_estimators=3, max_depth=3,max_leaf_nodes=10, 
                             max_features="auto", random_state=1)
clf.fit(X_train, y_train)
X_test_prediction = clf.predict(X_test)

labelAndprediction = zip(list(y_test),list(X_test_prediction))

recallBase = 0
recallUper = 0
precisionBase = 0
precisionUper = 0
accuracy = 0

for label,prediction in labelAndprediction:
    label = int(label)
    if label == 1 :
        recallBase += 1
        if prediction == 1 :
            recallUper += 1
recall = float(recallUper)/float(recallBase)

for label,prediction in labelAndprediction:
    prediction = int(prediction)
    if prediction == 1 :
        precisionBase += 1
        if label == 1 :
            precisionUper += 1
precision = float(precisionUper)/float(precisionBase)

for label,prediction in labelAndprediction:
    if label == prediction:
        accuracy += 1
accuracy = float(accuracy)/float(len(y_test))
f1score = (2*precision*recall)/(precision+recall)

print "accuracy : %f " %(accuracy)
print "recallBase : %f , recallUper : %f , recall : %f " %(recallBase, recallUper, recall)
print "precisionBase : %f , precisionUper : %f , precision : %f " %(precisionBase, precisionUper, precision)
print "f1score : %f " %(f1score)

accuracy : 0.973684 
recallBase : 12.000000 , recallUper : 12.000000 , recall : 1.000000 
precisionBase : 13.000000 , precisionUper : 12.000000 , precision : 0.923077 
f1score : 0.960000 


In [44]:
iris.data.shape

(150, 4)

In [45]:
from pyspark.mllib.regression import LabeledPoint
distData1 = sc.parallelize(iris.data)
distData2 = sc.parallelize(iris.target)
distData = distData2.zip(distData1).map(lambda x : LabeledPoint(x[0],x[1]))

In [46]:
distData.first()

LabeledPoint(0.0, [5.1,3.5,1.4,0.2])

In [47]:
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils

(trainingData, testData) = distData.randomSplit([0.75, 0.25])


model = RandomForest.trainClassifier(trainingData, numClasses=4, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=3, maxBins=10)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())


In [48]:
y_test = labelsAndPredictions.map(lambda x : x[0]).collect()
X_test_prediction = labelsAndPredictions.map(lambda x : x[1]).collect()

In [49]:
labelAndprediction = zip(list(y_test),list(X_test_prediction))

recallBase = 0
recallUper = 0
precisionBase = 0
precisionUper = 0
accuracy = 0

for label,prediction in labelAndprediction:
    label = int(label)
    if label == 1 :
        recallBase += 1
        if prediction == 1 :
            recallUper += 1
recall = float(recallUper)/float(recallBase)

for label,prediction in labelAndprediction:
    prediction = int(prediction)
    if prediction == 1 :
        precisionBase += 1
        if label == 1 :
            precisionUper += 1
precision = float(precisionUper)/float(precisionBase)

for label,prediction in labelAndprediction:
    if label == prediction:
        accuracy += 1
accuracy = float(accuracy)/float(len(y_test))
f1score = (2*precision*recall)/(precision+recall)

print "accuracy : %f " %(accuracy)
print "recallBase : %f , recallUper : %f , recall : %f " %(recallBase, recallUper, recall)
print "precisionBase : %f , precisionUper : %f , precision : %f " %(precisionBase, precisionUper, precision)
print "f1score : %f " %(f1score)

accuracy : 0.933333 
recallBase : 10.000000 , recallUper : 10.000000 , recall : 1.000000 
precisionBase : 12.000000 , precisionUper : 10.000000 , precision : 0.833333 
f1score : 0.909091 
